## Wprowadzenie

Ta lekcja obejmuje:
- Czym jest wywoływanie funkcji i do czego się je stosuje
- Jak utworzyć wywołanie funkcji za pomocą OpenAI
- Jak zintegrować wywołanie funkcji z aplikacją

## Cele nauki

Po ukończeniu tej lekcji będziesz wiedzieć i rozumieć:

- Cel stosowania wywoływania funkcji
- Konfigurowanie wywołania funkcji przy użyciu usługi OpenAI
- Projektowanie skutecznych wywołań funkcji dla zastosowań w Twojej aplikacji


## Zrozumienie wywołań funkcji

W tej lekcji chcemy stworzyć funkcję dla naszego startupu edukacyjnego, która pozwoli użytkownikom korzystać z chatbota do wyszukiwania kursów technicznych. Będziemy polecać kursy dopasowane do poziomu umiejętności użytkownika, jego obecnej roli oraz interesującej go technologii.

Aby to zrealizować, użyjemy połączenia:
 - `OpenAI` do stworzenia doświadczenia czatu dla użytkownika
 - `Microsoft Learn Catalog API`, aby pomóc użytkownikom znaleźć kursy na podstawie ich zapytań
 - `Function Calling`, aby pobrać zapytanie użytkownika i przekazać je do funkcji, która wykona żądanie do API

Na początek przyjrzyjmy się, dlaczego w ogóle warto korzystać z wywołań funkcji:

print("Wiadomości w następnym żądaniu:")
print(messages)
print()

second_response = client.chat.completions.create(
    messages=messages,
    model=deployment,
    function_call="auto",
    functions=functions,
    temperature=0
        )  # uzyskaj nową odpowiedź od GPT, która może zobaczyć odpowiedź funkcji


print(second_response.choices[0].message)


### Dlaczego wywoływanie funkcji

Jeśli ukończyłeś już którąkolwiek z innych lekcji w tym kursie, prawdopodobnie rozumiesz, jak potężne są duże modele językowe (LLM). Mam nadzieję, że dostrzegasz też ich pewne ograniczenia.

Wywoływanie funkcji to funkcja w usłudze OpenAI, która została stworzona, by rozwiązać następujące problemy:

Niespójny format odpowiedzi:
- Przed wprowadzeniem wywoływania funkcji odpowiedzi generowane przez duże modele językowe były nieustrukturyzowane i niespójne. Programiści musieli pisać skomplikowany kod walidujący, by obsłużyć każdą możliwą wariację odpowiedzi.

Ograniczona integracja z danymi zewnętrznymi:
- Wcześniej trudno było włączyć dane z innych części aplikacji do kontekstu rozmowy.

Dzięki standaryzacji formatów odpowiedzi i łatwej integracji z danymi zewnętrznymi, wywoływanie funkcji upraszcza proces tworzenia aplikacji i zmniejsza potrzebę pisania dodatkowej logiki walidującej.

Użytkownicy nie mogli uzyskać odpowiedzi na pytania typu „Jaka jest aktualna pogoda w Sztokholmie?”. Wynikało to z faktu, że modele były ograniczone do czasu, w którym zostały wytrenowane.

Przyjrzyjmy się poniższemu przykładowi, który ilustruje ten problem:

Załóżmy, że chcemy stworzyć bazę danych studentów, aby móc proponować im odpowiednie kursy. Poniżej znajdują się dwa opisy studentów, które zawierają bardzo podobne dane.


In [ ]:
student_1_description="Emily Johnson is a sophomore majoring in computer science at Duke University. She has a 3.7 GPA. Emily is an active member of the university's Chess Club and Debate Team. She hopes to pursue a career in software engineering after graduating."
 
student_2_description = "Michael Lee is a sophomore majoring in computer science at Stanford University. He has a 3.8 GPA. Michael is known for his programming skills and is an active member of the university's Robotics Club. He hopes to pursue a career in artificial intelligence after finshing his studies."

Chcemy wysłać to do LLM, aby przeanalizować dane. Później można to wykorzystać w naszej aplikacji do wysyłania do API lub przechowywania w bazie danych.

Stwórzmy dwa identyczne prompt’y, w których poinstruujemy LLM, jakie informacje nas interesują:


Chcemy wysłać to do LLM, aby przeanalizował części istotne dla naszego produktu. Dzięki temu możemy stworzyć dwa identyczne prompt'y, aby poinstruować LLM:


In [ ]:
prompt1 = f'''
Please extract the following information from the given text and return it as a JSON object:

name
major
school
grades
club

This is the body of text to extract the information from:
{student_1_description}
'''


prompt2 = f'''
Please extract the following information from the given text and return it as a JSON object:

name
major
school
grades
club

This is the body of text to extract the information from:
{student_2_description}
'''


Po utworzeniu tych dwóch promptów, wyślemy je do LLM za pomocą `openai.ChatCompletion`. Przechowujemy prompt w zmiennej `messages` i przypisujemy rolę `user`. Ma to na celu naśladowanie wiadomości od użytkownika pisanej do chatbota.


In [ ]:
import os
import json
from openai import OpenAI
from dotenv import load_dotenv
load_dotenv()

client = OpenAI()

deployment="gpt-3.5-turbo"

: 

In [ ]:
openai_response1 = client.chat.completions.create(
 model=deployment,    
 messages = [{'role': 'user', 'content': prompt1}]
)
openai_response1.choices[0].message.content 

In [ ]:
openai_response2 = client.chat.completions.create(
 model=deployment,    
 messages = [{'role': 'user', 'content': prompt2}]
)
openai_response2.choices[0].message.content

In [ ]:
# Loading the response as a JSON object
json_response1 = json.loads(openai_response1.choices[0].message.content)
json_response1

In [ ]:
# Loading the response as a JSON object
json_response2 = json.loads(openai_response2.choices[0].message.content )
json_response2

Nawet jeśli polecenia są takie same, a opisy podobne, możemy otrzymać różne formaty właściwości `Grades`.

Jeśli uruchomisz powyższą komórkę kilka razy, format może być `3.7` lub `3.7 GPA`.

Dzieje się tak, ponieważ LLM przyjmuje nieustrukturyzowane dane w postaci napisanego polecenia i zwraca również dane nieustrukturyzowane. Potrzebujemy mieć ustrukturyzowany format, aby wiedzieć, czego się spodziewać podczas przechowywania lub używania tych danych.

Korzystając z wywołań funkcji, możemy mieć pewność, że otrzymamy dane w ustrukturyzowanej formie. Podczas korzystania z wywołań funkcji LLM faktycznie nie wywołuje ani nie uruchamia żadnych funkcji. Zamiast tego tworzymy strukturę, której LLM ma się trzymać w swoich odpowiedziach. Następnie używamy tych ustrukturyzowanych odpowiedzi, aby wiedzieć, jaką funkcję uruchomić w naszych aplikacjach.


![Schemat przepływu wywołań funkcji](../../../../translated_images/Function-Flow.083875364af4f4bb69bd6f6ed94096a836453183a71cf22388f50310ad6404de.pl.png)


### Przykłady użycia wywołań funkcji

**Wywoływanie narzędzi zewnętrznych**  
Chatboty świetnie odpowiadają na pytania użytkowników. Dzięki wywołaniom funkcji chatboty mogą wykorzystywać wiadomości od użytkowników do realizacji określonych zadań. Na przykład student może poprosić chatbota: "Wyślij e-mail do mojego wykładowcy z informacją, że potrzebuję więcej pomocy z tym tematem". Wtedy może zostać wywołana funkcja `send_email(to: string, body: string)`

**Tworzenie zapytań do API lub bazy danych**  
Użytkownicy mogą wyszukiwać informacje za pomocą języka naturalnego, który zostaje przekształcony w odpowiednie zapytanie lub żądanie do API. Przykładem może być nauczyciel, który pyta: "Którzy uczniowie ukończyli ostatnie zadanie?", co może wywołać funkcję o nazwie `get_completed(student_name: string, assignment: int, current_status: string)`

**Tworzenie danych strukturalnych**  
Użytkownicy mogą wziąć fragment tekstu lub plik CSV i wykorzystać LLM do wyodrębnienia z niego najważniejszych informacji. Na przykład student może przekształcić artykuł z Wikipedii o porozumieniach pokojowych, aby stworzyć fiszki AI. Można to zrobić za pomocą funkcji `get_important_facts(agreement_name: string, date_signed: string, parties_involved: list)`


## 2. Tworzenie pierwszego wywołania funkcji

Proces tworzenia wywołania funkcji obejmuje 3 główne kroki:
1. Wywołanie Chat Completions API z listą Twoich funkcji oraz wiadomością od użytkownika
2. Odczytanie odpowiedzi modelu, aby wykonać jakąś akcję, np. wywołać funkcję lub API
3. Ponowne wywołanie Chat Completions API z odpowiedzią z Twojej funkcji, aby wykorzystać te informacje do stworzenia odpowiedzi dla użytkownika.


![Przepływ wywołania funkcji](../../../../translated_images/LLM-Flow.3285ed8caf4796d7343c02927f52c9d32df59e790f6e440568e2e951f6ffa5fd.pl.png)


### Elementy wywołania funkcji

#### Wprowadzenie przez użytkownika

Pierwszym krokiem jest utworzenie wiadomości od użytkownika. Można to zrobić dynamicznie, pobierając wartość z pola tekstowego, lub przypisać wartość bezpośrednio tutaj. Jeśli to Twój pierwszy kontakt z API Chat Completions, musimy określić `role` oraz `content` wiadomości.

`role` może przyjmować wartości: `system` (tworzenie zasad), `assistant` (model) lub `user` (użytkownik końcowy). W przypadku wywoływania funkcji przypiszemy rolę `user` oraz przykładowe pytanie.


In [ ]:
messages= [ {"role": "user", "content": "Find me a good course for a beginner student to learn Azure."} ]

### Tworzenie funkcji.

Teraz zdefiniujemy funkcję oraz jej parametry. W tym przykładzie użyjemy tylko jednej funkcji o nazwie `search_courses`, ale możesz stworzyć ich więcej.

**Ważne**: Funkcje są dołączane do wiadomości systemowej dla LLM i będą wliczane do dostępnej liczby tokenów.


In [ ]:
functions = [
   {
      "name":"search_courses",
      "description":"Retrieves courses from the search index based on the parameters provided",
      "parameters":{
         "type":"object",
         "properties":{
            "role":{
               "type":"string",
               "description":"The role of the learner (i.e. developer, data scientist, student, etc.)"
            },
            "product":{
               "type":"string",
               "description":"The product that the lesson is covering (i.e. Azure, Power BI, etc.)"
            },
            "level":{
               "type":"string",
               "description":"The level of experience the learner has prior to taking the course (i.e. beginner, intermediate, advanced)"
            }
         },
         "required":[
            "role"
         ]
      }
   }
]

**Definicje**

Struktura definicji funkcji ma kilka poziomów, z których każdy posiada własne właściwości. Oto szczegółowy opis tej zagnieżdżonej struktury:

**Właściwości funkcji na najwyższym poziomie:**

`name` - Nazwa funkcji, którą chcemy wywołać.

`description` - Opis działania funkcji. Ważne, aby był precyzyjny i jasny.

`parameters` - Lista wartości oraz format, które model ma wygenerować w odpowiedzi.

**Właściwości obiektu Parameters:**

`type` - Typ danych obiektu parameters (zazwyczaj "object")

`properties` - Lista konkretnych wartości, które model wykorzysta w odpowiedzi

**Właściwości pojedynczego parametru:**

`name` - Domyślnie określone przez klucz właściwości (np. "role", "product", "level")

`type` - Typ danych tego konkretnego parametru (np. "string", "number", "boolean")

`description` - Opis konkretnego parametru

**Właściwości opcjonalne:**

`required` - Tablica z nazwami parametrów, które są wymagane do poprawnego wywołania funkcji


### Wywoływanie funkcji
Po zdefiniowaniu funkcji musimy teraz uwzględnić ją w wywołaniu do Chat Completion API. Robimy to, dodając `functions` do zapytania. W tym przypadku `functions=functions`.

Jest także opcja ustawienia `function_call` na `auto`. Oznacza to, że pozwolimy LLM zdecydować, która funkcja powinna zostać wywołana na podstawie wiadomości użytkownika, zamiast przypisywać ją samodzielnie.


In [ ]:
response = client.chat.completions.create(model=deployment, 
                                        messages=messages,
                                        functions=functions, 
                                        function_call="auto") 

print(response.choices[0].message)

Teraz przyjrzyjmy się odpowiedzi i zobaczmy, jak jest sformatowana:

{
  "role": "assistant",
  "function_call": {
    "name": "search_courses",
    "arguments": "{\n  \"role\": \"student\",\n  \"product\": \"Azure\",\n  \"level\": \"beginner\"\n}"
  }
}

Widzisz, że wywoływana jest nazwa funkcji, a na podstawie wiadomości użytkownika LLM był w stanie znaleźć dane pasujące do argumentów funkcji.


## 3. Integracja wywołań funkcji z aplikacją.

Po przetestowaniu sformatowanej odpowiedzi z LLM, możemy teraz zintegrować ją z aplikacją.

### Zarządzanie przepływem

Aby zintegrować to z naszą aplikacją, wykonajmy następujące kroki:

Najpierw wywołajmy usługi OpenAI i zapiszmy wiadomość w zmiennej o nazwie `response_message`.


In [ ]:
response_message = response.choices[0].message

Teraz zdefiniujemy funkcję, która wywoła API Microsoft Learn, aby pobrać listę kursów:


In [ ]:
import requests

def search_courses(role, product, level):
    url = "https://learn.microsoft.com/api/catalog/"
    params = {
        "role": role,
        "product": product,
        "level": level
    }
    response = requests.get(url, params=params)
    modules = response.json()["modules"]
    results = []
    for module in modules[:5]:
        title = module["title"]
        url = module["url"]
        results.append({"title": title, "url": url})
    return str(results)



Jako dobra praktyka, sprawdzimy, czy model chce wywołać jakąś funkcję. Następnie utworzymy jedną z dostępnych funkcji i dopasujemy ją do tej, która jest wywoływana. 
Następnie weźmiemy argumenty funkcji i przypiszemy je do argumentów pochodzących z LLM.

Na końcu dołączymy wiadomość o wywołaniu funkcji oraz wartości zwrócone przez wiadomość `search_courses`. Dzięki temu LLM będzie miał wszystkie potrzebne informacje,
aby odpowiedzieć użytkownikowi w naturalny sposób.


In [ ]:
# Check if the model wants to call a function
if response_message.function_call.name:
    print("Recommended Function call:")
    print(response_message.function_call.name)
    print()

    # Call the function. 
    function_name = response_message.function_call.name

    available_functions = {
            "search_courses": search_courses,
    }
    function_to_call = available_functions[function_name] 

    function_args = json.loads(response_message.function_call.arguments)
    function_response = function_to_call(**function_args)

    print("Output of function call:")
    print(function_response)
    print(type(function_response))


    # Add the assistant response and function response to the messages
    messages.append( # adding assistant response to messages
        {
            "role": response_message.role,
            "function_call": {
                "name": function_name,
                "arguments": response_message.function_call.arguments,
            },
            "content": None
        }
    )
    messages.append( # adding function response to messages
        {
            "role": "function",
            "name": function_name,
            "content":function_response,
        }
    )



In [ ]:
print("Messages in next request:")
print(messages)
print()

second_response = client.chat.completions.create(
    messages=messages,
    model=deployment,
    function_call="auto",
    functions=functions,
    temperature=0
        )  # get a new response from GPT where it can see the function response


print(second_response.choices[0].message)

## Wyzwanie programistyczne

Świetna robota! Aby dalej rozwijać swoją wiedzę na temat wywoływania funkcji OpenAI, możesz zbudować: https://learn.microsoft.com/training/support/catalog-api-developer-reference?WT.mc_id=academic-105485-koreyst
 - Więcej parametrów funkcji, które mogą pomóc użytkownikom znaleźć więcej kursów. Dostępne parametry API znajdziesz tutaj:
 - Stwórz kolejne wywołanie funkcji, które pobiera więcej informacji od użytkownika, na przykład jego język ojczysty
 - Dodaj obsługę błędów na wypadek, gdyby wywołanie funkcji i/lub API nie zwróciło żadnych odpowiednich kursów



---

**Zastrzeżenie**:  
Ten dokument został przetłumaczony przy użyciu usługi tłumaczenia AI [Co-op Translator](https://github.com/Azure/co-op-translator). Chociaż dokładamy wszelkich starań, aby tłumaczenie było poprawne, prosimy pamiętać, że automatyczne tłumaczenia mogą zawierać błędy lub nieścisłości. Oryginalny dokument w jego rodzimym języku powinien być traktowany jako źródło nadrzędne. W przypadku informacji krytycznych zalecane jest skorzystanie z profesjonalnych usług tłumacza. Nie ponosimy odpowiedzialności za jakiekolwiek nieporozumienia lub błędne interpretacje wynikające z korzystania z tego tłumaczenia.
